In [2]:
from owlready2 import *

onto1 = get_ontology("file:///home/michael/Desktop/CrosetWork/t3test2_no_semicolon_G").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [36]:
#assign appropriate labels to classes

x = list(onto1.classes())
for y in x:
    term = 'GO:' + str(y)[-7:]
    if str(y)[-8] == 'G':
        y.label = nary[('GO:' + str(y)[-7:])]['name']

In [46]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto1.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = 'GO:' + cl[-7:]
    ids.append(clt)

In [40]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

#remove anything that has no aspect or is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value.get('aspect') == None:
            keys.append(key)
            c+=1
    elif value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
        
print(c)

for key, value in nary.items():
    for i in value['children']:
        if i['id'] in keys:
            c+=1
        elif value.get('babies') == None:
            value['babies'] = []
            value['babies'].append(i)
        else:
            value['babies'].append(i)
for i in keys:
    del nary[i]

4367


In [41]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto1:
            classname = types.new_class((pname), (ObjectProperty,), )
    return 0

In [1]:
#parsing biological process and molecular function classes

#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

reldict = {}

for key, value in kilnary.items():            
    for item in value['children']:
        if reldict.get(item['relation']) == None:
            reldict[item['relation']] = [[],[]]
            propertymaker(item['relation'], 'void', 'void')
    for item in value['children']:
        if item['id'] in ids:
            reldict[item['relation']][1].append(ids[ids.index(value['id'])])
            reldict[item['relation']][0].append(ids[ids.index(item['id'])])

NameError: name 'nary' is not defined

In [52]:
#Making subproperties positively_regulates and negatively_regulates

with onto1:
    class regulates(ObjectProperty):
        pass
    class positively_regulates(regulates):
        pass
    class negatively_regulates(regulates):
        pass

In [53]:
#converting string to object in reldict lists

for key, value in reldict.items():
    for item in value[0]:
        value[0][value[0].index(item)] = getattr(onto1, ('G' + item[-7:]))
    for item in value[1]:
        value[1][value[1].index(item)] = getattr(onto1, ('G' + item[-7:]))


In [56]:
#setting relations

for key, value in reldict.items():
    for item in value[0]:
        setattr(item, key, [value[1][value[0].index(item)]])

In [57]:
onto1.save(file='t3test2_no_semicolon_G_labelled_related.owl', format = 'rdfxml')